In [1]:
from __future__ import annotations

from pathlib import Path
import pandas as pd
import logging

pyradiomics_logger = logging.getLogger("radiomics")
pyradiomics_logger.setLevel(logging.ERROR)

# Setup and Configuration

In [2]:
# a Save data to local directory
DATA_DIR = Path("data")

# If you choose a different collection in the setup notebook, you will need to change this value
COLLECTION_ID = "nsclc_radiomics"

NIFTI_OUTPUT_DIR = DATA_DIR / "images" / COLLECTION_ID / "niftis"

PYRADIOMICS_CONFIG = Path().cwd().parent / "pyradiomics.yaml"

IMAGE_TYPES = [
    "shuffled_full",
    "shuffled_roi",
    "shuffled_non_roi",
    "randomized_sampled_full",
    "randomized_sampled_roi",
    "randomized_sampled_non_roi",
]

In [3]:
! tree -F $NIFTI_OUTPUT_DIR

data/images/nsclc_radiomics/niftis/
├── SubjectID-0_LUNG1-175/
│   ├── CT_SeriesUID-51021/
│   │   ├── original.nii.gz
│   │   ├── randomized_sampled_full.nii.gz
│   │   ├── randomized_sampled_non_roi.nii.gz
│   │   ├── randomized_sampled_roi.nii.gz
│   │   ├── shuffled_full.nii.gz
│   │   ├── shuffled_non_roi.nii.gz
│   │   └── shuffled_roi.nii.gz
│   └── RTSTRUCT_SeriesUID-85917/
│       └── GTV.nii.gz
└── dataset_index.csv

4 directories, 9 files


In [4]:
file_df = pd.read_csv(NIFTI_OUTPUT_DIR / 'dataset_index.csv')
file_df

,PatientID,Modality,SeriesInstanceUID,IMAGE_ID,filepath
0,0_LUNG1-175,CT,51021,randomized_sampled_non_roi,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/rando...
1,0_LUNG1-175,CT,51021,shuffled_full,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/shuff...
2,0_LUNG1-175,CT,51021,randomized_sampled_roi,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/rando...
3,0_LUNG1-175,CT,51021,original,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/origi...
4,0_LUNG1-175,CT,51021,randomized_sampled_full,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/rando...
5,0_LUNG1-175,CT,51021,shuffled_non_roi,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/shuff...
6,0_LUNG1-175,CT,51021,shuffled_roi,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/shuff...
7,0_LUNG1-175,RTSTRUCT,85917,GTV,SubjectID-0_LUNG1-175/RTSTRUCT_SeriesUID-85917...


# Run Single Radiomic Feature Extraction


In [55]:
from readii.feature_extraction import singleRadiomicFeatureExtraction
from SimpleITK import ReadImage

features = []

for patient_id, group in file_df.groupby(["PatientID"], sort=False):
    rt = group.query('Modality=="RTSTRUCT"').filepath.values[0]
    rt_image = ReadImage(NIFTI_OUTPUT_DIR / rt)
    for neg in group.query('Modality=="CT"').itertuples():
        tuple_as_dict = neg._asdict()
        ct = ReadImage(NIFTI_OUTPUT_DIR / neg.filepath)

        orderd_dictionary = singleRadiomicFeatureExtraction(ct, rt_image, str(PYRADIOMICS_CONFIG))
        tuple_as_dict.update(orderd_dictionary)
        features.append(tuple_as_dict)



In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average =

KeyboardInterrupt: 

In [18]:
pd.DataFrame(features)

,Index,PatientID,Modality,SeriesInstanceUID,IMAGE_ID,filepath,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,...,gradient_gldm_LargeDependenceLowGrayLevelEmphasis,gradient_gldm_LowGrayLevelEmphasis,gradient_gldm_SmallDependenceEmphasis,gradient_gldm_SmallDependenceHighGrayLevelEmphasis,gradient_gldm_SmallDependenceLowGrayLevelEmphasis,gradient_ngtdm_Busyness,gradient_ngtdm_Coarseness,gradient_ngtdm_Complexity,gradient_ngtdm_Contrast,gradient_ngtdm_Strength
0,0,0_LUNG1-175,CT,51021,randomized_sampled_non_roi,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/rando...,v3.1.4,1.26.4,2.4.0,1.8.0,...,159.2979125500727,0.3811807153037106,0.1796243439764029,40.198851277043616,0.006645854524640535,20.90751673889937,0.0002862000824514884,1390.781524716655,0.10371764267278706,0.4299214341828308
1,1,0_LUNG1-175,CT,51021,shuffled_full,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/shuff...,v3.1.4,1.26.4,2.4.0,1.8.0,...,0.06696013163625358,0.01304073347296209,0.453101708874787,138.23699566359923,0.007134978245334792,6.147555966183211,0.00018562721939375311,6665.993365703826,0.17801377600841764,0.2568787224881177
2,2,0_LUNG1-175,CT,51021,randomized_sampled_roi,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/rando...,v3.1.4,1.26.4,2.4.0,1.8.0,...,1.9793193317317448,0.07765606653795411,0.1745243713854,13.633031505187699,0.01643843952845416,28.107960885826284,0.00019742040475273672,732.0516457014744,0.059377005924186495,0.10449057874400199
3,3,0_LUNG1-175,CT,51021,original,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/origi...,v3.1.4,1.26.4,2.4.0,1.8.0,...,196.9183855548549,0.46266530392709343,0.06408276482325558,3.557415180885961,0.005844256986665003,39.79074192044453,0.00033157321935797226,213.75465014081112,0.028485529131768033,0.2139720935160493
4,4,0_LUNG1-175,CT,51021,randomized_sampled_full,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/rando...,v3.1.4,1.26.4,2.4.0,1.8.0,...,0.06637795630487293,0.012929536740815019,0.4484693390727418,138.0121775304685,0.007145252196131746,7.614899359898904,0.00018629552777701174,4599.876401672243,0.22522959784858027,0.15557064744348403
5,5,0_LUNG1-175,CT,51021,shuffled_non_roi,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/shuff...,v3.1.4,1.26.4,2.4.0,1.8.0,...,158.3640828943188,0.38078638174840307,0.18008164004000143,38.93284406309167,0.0066736271703146735,18.489437506358666,0.00028513392066708984,1585.044455915988,0.09019742035971783,0.5143623810991863
6,6,0_LUNG1-175,CT,51021,shuffled_roi,SubjectID-0_LUNG1-175/CT_SeriesUID-51021/shuff...,v3.1.4,1.26.4,2.4.0,1.8.0,...,2.105760541342411,0.0803890796907509,0.17507744911728104,13.479985932217424,0.01640072543714562,28.20709393928059,0.00019746063700172342,737.9263002121309,0.058588272057694166,0.1065495641675521


# Run FMCIB Radiomic Feature Extraction


In [75]:
from fmcib.preprocessing import preprocess, get_transforms
from fmcib.models import resnet50, get_linear_classifier, LoadModel, fmcib_model
import SimpleITK as sitk
import numpy as np
from imgtools.ops import Resize

def find_bbox(mask: sitk.Image) -> np.ndarray:
    """
    Finds the bounding box of a given mask image.

    Parameters:
    mask (sitk.Image): The input mask image.

    Returns:
    np.ndarray: The bounding box coordinates as a numpy array, [xstart, xend, ystart, yend, zstart, zend].
    """
    mask_uint = sitk.Cast(mask, sitk.sitkUInt8)
    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(mask_uint)
    xstart, ystart, zstart, xsize, ysize, zsize = stats.GetBoundingBox(1)
    
    # Prevent the following ITK Error from SmoothingRecursiveGaussianImageFilter: 
    # The number of pixels along dimension 2 is less than 4. This filter requires a minimum of four pixels along the dimension to be processed.
    if xsize < 4:
        xsize = 4
    if ysize < 4:
        ysize = 4
    if zsize < 4:
        zsize = 4

    xend, yend, zend = xstart + xsize, ystart + ysize, zstart + zsize
    return xstart, xend, ystart, yend, zstart, zend

def crop_bbox(image: sitk.Image, bbox_coords: tuple, input_size: tuple) -> sitk.Image:
    """
    Crops a bounding box from the given image and resizes it to the specified input size.
    The if/else statements are used to ensure that the bounding box is not cropped outside the image boundaries.

    Args:
        image (sitk.Image): The input image from which the bounding box will be cropped.
        bbox_coords (tuple): Cordinates of the bounding box.
        input_size (tuple): Desired output size of the cropped image. eg. (50, 50, 50)
    Returns:
        sitk.Image: The cropped and resized image.
    """
    min_x, max_x, min_y, max_y, min_z, max_z = bbox_coords
    img_x, img_y, img_z = image.GetSize()

    if min_x < 0: 
        min_x, max_x = 0, input_size[0]
    elif max_x > img_x: # input_size[0]:
        min_x, max_x = img_x - input_size[0], img_x

    if min_y < 0:
        min_y, max_y = 0, input_size[1]
    elif max_y > img_y: # input_size[1]:
        min_y, max_y = img_y - input_size[1], img_y

    if min_z < 0:
        min_z, max_z = 0, input_size[2]
    elif max_z > img_z: # input_size[2]:
        min_z, max_z = img_z - input_size[2], img_z
    
    img_crop = image[min_x:max_x, min_y:max_y, min_z:max_z]
    img_crop = Resize(input_size)(img_crop)
    return img_crop

In [82]:
trunk = resnet50(
            pretrained=False,
            n_input_channels=1,
            widen_factor=2,
            conv1_t_stride=2,
            feed_forward=False,
            bias_downsample=True,
        )
model = LoadModel(trunk=trunk, weights_path='/Users/bhklab/dev/radiomics/readii-idc-notebooks/notebooks/model_weights.torch').to('cpu')



2024-12-19 17:40:13.247 | WARNING  | fmcib.models.load_model:load:104 - Missing keys: [] and unexpected keys: []
2024-12-19 17:40:13.248 | INFO     | fmcib.models.load_model:load:129 - Loaded pretrained model weights 



In [84]:
image = sitk.ReadImage(NIFTI_OUTPUT_DIR / neg.filepath)
rt_image = sitk.ReadImage(NIFTI_OUTPUT_DIR / rt)
cropped = crop_bbox(image, find_bbox(rt_image), (50, 50, 50))

newfile = sitk.WriteImage(cropped, "cropped.nii.gz")
cropped.GetSize()

(50, 50, 50)

In [106]:

lung1_row = {
    "image_path": str((NIFTI_OUTPUT_DIR / neg.filepath).parent / 'original.nii.gz'),
    'coordX': 0,
    'coordY': 0,
    'coordZ': 0,
}
print(lung1_row)

{'image_path': 'data/images/nsclc_radiomics/niftis/SubjectID-0_LUNG1-175/CT_SeriesUID-51021/original.nii.gz', 'coordX': 0, 'coordY': 0, 'coordZ': 0}


In [107]:
T = get_transforms(spatial_size=(50, 50, 50), precropped=True)
timage = T(lung1_row)


In [108]:
model.eval()

model(timage.unsqueeze(0)).detach().numpy()



array([[1.0495391 , 0.10860432, 0.52121043, ..., 3.0670488 , 2.7885752 ,
        1.8654134 ]], dtype=float32)